In [1]:
import netCDF4 as nc
from metpy.plots import USCOUNTIES
import metpy
import cartopy.crs as ccrs 
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import wrf
from wrf import vinterp, to_np, getvar, CoordPair, vertcross, get_cartopy
import pandas as pd
import matplotlib.ticker as ticker
import os
from metpy.units import units 
import metpy.calc as mpcalc

C:\Users\Tony\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
def cape_cross(ncfile, number):
    ncfile = nc.Dataset(ncfile)
    t = getvar(ncfile, "temp", units='k') * units.kelvin
    td = getvar(ncfile, "td", units='k') * units.kelvin
    p = getvar(ncfile, "pressure") * units.hPa

    num_lowest_levels = number
    num_bottom_top = t.shape[0]
    num_south_north = t.shape[1]
    num_west_east = t.shape[2]

    sbcape_array_3d = np.zeros((num_lowest_levels, num_south_north, num_west_east))
    sbcin_array_3d = np.zeros((num_lowest_levels, num_south_north, num_west_east))

    for level in range(num_lowest_levels):
        for i in range(num_south_north):
            for j in range(num_west_east):
                profile_t = t[level:, i, j].values * units.kelvin
                profile_dew = td[level:, i, j].values * units.kelvin
                profile_p = p[level:, i, j].values * units.hPa

                cape, cin = mpcalc.surface_based_cape_cin(profile_p, profile_t, profile_dew)

                sbcape_array_3d[level, i, j] = cape.m
                sbcin_array_3d[level, i, j] = cin.m

    return sbcape_array_3d, sbcin_array_3d
    

In [3]:
sbcape, sbcin = cape_cross("c:\\Users\\Tony\\Desktop\\wrfoutput\\ctrl\\wrfout_d02_2019-07-20_05_30_00", 20)

C:\Users\Tony\AppData\Local\Temp\ipykernel_25464\3176956884.py:25: UserWarning: Interpolation point out of data bounds encountered
  cape, cin = mpcalc.surface_based_cape_cin(profile_p, profile_t, profile_dew)


In [4]:
file_path = 'C:\\Users\\Tony\\Desktop\\METR 4990\\7_20_05_30_sbcape_array_3d.npy'
file_path2 = 'C:\\Users\\Tony\\Desktop\\METR 4990\\7_20_05_30_sbcin_array_3d.npy'


np.save(file_path, sbcape)
np.save(file_path2, sbcin)